In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.constants import speed_of_light, elementary_charge, electron_mass, hbar as hbar_SI
import matplotlib.pyplot as plt

In [ ]:
me_SI = electron_mass
e_SI = elementary_charge
c_SI = speed_of_light

meV = e_SI * 1e-3
nm = 1e-9
ps = 1e-12

c = c_SI * ps / nm
hbar = hbar_SI / (meV * ps)
m = me_SI * c_SI ** 2 / meV / c ** 2

In [ ]:
omega = 2 / hbar
vQD = 1500000

In [ ]:
x_min, x_max = -50, 150
x0 = 0
x1 = 75

t1 = 2
t2 = t1 + (x1 - x0) / vQD

t_min, t_max = t1, t2

In [ ]:
dt = 0.000000001

t_values = np.arange(t_min, t_max, dt)
Nt = len(t_values)
x_values = np.linspace(x_min, x_max, Nt)

dx = x_values[1] - x_values[0]

laplacian = sp.diags([1, -2, 1], offsets=[-1, 0, 1], shape=(Nt, Nt), format='csc') / dx ** 2
H_kinetic = - (hbar ** 2 / (2 * m)) * laplacian

In [ ]:
skip = Nt // 2000
offset = Nt % 2000

if (Nt <= 2000):
  Ns = Nt
  offset = 0
  skip = 1
else:
  Ns = len(x_values[::skip])

print(f"N = {Nt}\nNs = {Ns}\nskip = {skip}\noffset = {offset}")

In [ ]:
def ground_state(x):
    A = (m * omega / (np.pi * hbar)) ** 0.25
    alpha = (m * omega) / (2.0 * hbar)
    return A * np.exp(-alpha * (x ** 2))

def first_state(x):
    A = 1 / np.sqrt(2)
    B = ((m * omega) / (np.pi * hbar)) ** 0.25
    C = np.exp((-m * omega * (x ** 2)) / (2 * hbar))
    D = 2 * np.sqrt((m * omega) / hbar) * x
    
    return A * B * C * D

In [ ]:
psi = ground_state(x_values)
norm = np.sqrt(np.sum(np.abs(psi)**2) * dx)
psi /= norm

In [ ]:
psi_real_analytical = np.zeros((Ns, Ns))
psi_img_analytical = np.zeros((Ns, Ns))
psi_real_analytical[:, 0] = np.real(psi[::skip])
psi_img_analytical[:, 0] = np.imag(psi[::skip])

In [ ]:
def quantum_center(t_values):
    xqd_arr = np.zeros_like(t_values)
    
    for i, t in enumerate(t_values):
        if t < t1:
            xqd_arr[i] = x0
        elif t < t2:
            xqd_arr[i] = x0 + vQD * (t - t1)
        else:
            xqd_arr[i] = x1
    return xqd_arr

In [ ]:
I = sp.eye(Nt, format='csc')
xqd_arr = quantum_center(t_values)

In [ ]:
for t_i in range(1, Nt):
	if t_i % 100 == 0:
		print(f"Step {t_i} / {Nt}")	

	V = 0.5 * m * omega ** 2 * (x_values - xqd_arr[t_i]) ** 2
	H = H_kinetic + sp.diags(V, 0, format='csc')

	A = I + 1j * dt / (2 * hbar) * H
	B = I - 1j * dt / (2 * hbar) * H

	psi = spla.spsolve(A, B @ psi)

	norm = np.sqrt(np.sum(np.abs(psi) ** 2) * dx)

	if norm > 0:
		psi /= norm
	else:
		print(f"Zero norm at step {t_i}")
		break

	if t_i % skip == 0:
		psi_real_analytical[:, t_i // skip] = np.real(psi[::skip])
		psi_img_analytical[:, t_i // skip] = np.imag(psi[::skip])

In [ ]:
plt.figure(figsize=(8, 6), dpi=200)
psi_sq = psi_real_analytical ** 2 + psi_img_analytical ** 2

plt.pcolormesh(x_values[::skip], t_values[::skip], psi_sq.transpose(), shading="auto")
plt.colorbar(label='|psi(x,t)|^2')
plt.xlabel('x')
plt.ylabel('t')
plt.title(f'Numerical |psi|^2 --- vQD = {vQD} --- dt = {dt}')
plt.show()

In [ ]:
# from scipy.integrate import simpson
# from scipy import special

# psi_pinn = psi_real_analytical + 1j * psi_img_analytical

# def quantum_center(t):
#     if t < t1:
#         return x0
#     elif t < t1 + (x1 - x0) / vQD:
#         return x0 + vQD * (t - t1)
#     else:
#         return x1
    
# def psi_x_n(x, t, n):
#     A = 1 / np.sqrt((2 ** n) * special.factorial(n))
#     B = ((m * omega) / (np.pi * hbar)) ** (1/4)
#     C = np.exp((-m * omega * (x - quantum_center(t)) ** 2) / (2 * hbar))
#     D = special.eval_hermite(n, np.sqrt((m * omega) / hbar) * (x - quantum_center(t)))
#     return A * B * C * D

# def psi_x_t_n(x, t, n):
#     A = psi_x_n(x, t, n)
#     En = (n + (1/2)) * hbar * omega
#     B = np.exp(-(1j * En * t) / hbar)
#     return A * B

# lambda0, lambda1 = [], []
# lambda2, lambda3 = [], []
# lambda4, lambda5 = [], []

# for i, t in enumerate(t_values):
#     psi = psi_pinn[:, i]
    
#     psi0_t = psi_x_t_n(x_values, t, 0)
#     psi1_t = psi_x_t_n(x_values, t, 1)
#     psi2_t = psi_x_t_n(x_values, t, 2)
#     psi3_t = psi_x_t_n(x_values, t, 3)
#     psi4_t = psi_x_t_n(x_values, t, 4)
#     psi5_t = psi_x_t_n(x_values, t, 5)
    
#     lambda0_i = simpson(psi * np.conj(psi0_t), x=x_values)
#     lambda1_i = simpson(psi * np.conj(psi1_t), x=x_values)
#     lambda2_i = simpson(psi * np.conj(psi2_t), x=x_values)
#     lambda3_i = simpson(psi * np.conj(psi3_t), x=x_values)
#     lambda4_i = simpson(psi * np.conj(psi4_t), x=x_values)
#     lambda5_i = simpson(psi * np.conj(psi5_t), x=x_values)
    
#     lambda0.append(np.abs(lambda0_i) ** 2)
#     lambda1.append(np.abs(lambda1_i) ** 2)
#     lambda2.append(np.abs(lambda2_i) ** 2)
#     lambda3.append(np.abs(lambda3_i) ** 2)
#     lambda4.append(np.abs(lambda4_i) ** 2)
#     lambda5.append(np.abs(lambda5_i) ** 2)

# plt.figure(figsize=(8,5), dpi=200)
# plt.plot(t_values, lambda0, label='|lambda0(t)| (ground state)')
# plt.plot(t_values, lambda1, label='|lambda1(t)| (first excited)')
# plt.plot(t_values, lambda2, label='|lambda2(t)| (second excited)')
# plt.plot(t_values, lambda3, label='|lambda3(t)| (third excited)')
# plt.plot(t_values, lambda4, label='|lambda4(t)| (forth excited)')
# plt.plot(t_values, lambda5, label='|lambda5(t)| (fifth excited)')
# plt.xlabel("Time (ps)")
# plt.ylabel("Probability weight")
# plt.title("Time-dependent weights of ground and first excited states")
# plt.legend()
# plt.grid(True)
# plt.show()